# Sample Mass-Difference Networks in Metabolomics Data Analysis

Notebook to support the study on the application of **Sample M**ass-**Di**fference **N**etworks as a highly specific competing form of pre-processing procedure for high-resolution metabolomics data.

Mass-Difference Networks are focused into making networks from a list of masses. Each _m/z_ will represent a node. Nodes will be connected if the difference in their masses can be associated to a simple chemical reaction (enzymatic or non-enzymatic) that led to a change in the elemental composition of its metabolite.

The set of mass differences used to build said networks are called a set of MDBs - Mass-Difference-based Building block.

This is notebook `paper_sMDiNs_database_prep.ipynb`.


## Organization of the Notebook

- **Set up database of the benchmark datasets**
- Description of the benchmark datasets
- **Building transformation list (of MDBs) and setting up list of masses files to use in Cytoscape's MetaNetter (to build MDiNs)**
- Show the benchmark datasets characteristics
- Explanation of the parameters used in Cytoscape's MetaNetter 2.0 to build the MDiNs.
- Apply intensity-based pre-treatments.
- **Store pre-processed and pre-treated data**


#### Needed Imports

In [ ]:
import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal

# Metabolinks package
import metabolinks as mtl
import metabolinks.transformations as transf

from pathlib import Path
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor

# json for persistence
import json
from time import perf_counter

In [ ]:
%matplotlib inline

In [ ]:
# Atomic masses - https://ciaaw.org/atomic-masses.htm
#Isotopic abundances-https://ciaaw.org/isotopic-abundances.htm/https://www.degruyter.com/view/journals/pac/88/3/article-p293.xml
# Isotopic abundances from Pure Appl. Chem. 2016; 88(3): 293–306,
# Isotopic compositions of the elements 2013 (IUPAC Technical Report), doi: 10.1515/pac-2015-0503

chemdict = {'H':(1.0078250322, 0.999844),
            'C':(12.000000000, 0.988922),
            'N':(14.003074004, 0.996337),
            'O':(15.994914619, 0.9976206),
            'Na':(22.98976928, 1.0),
            'P':(30.973761998, 1.0),
            'S':(31.972071174, 0.9504074),
            'Cl':(34.9688527, 0.757647),
            'F':(18.998403163, 1.0),
            'C13':(13.003354835, 0.011078) # Carbon 13 isotope
           } 

# electron mass from NIST http://physics.nist.gov/cgi-bin/cuu/Value?meu|search_for=electron+mass
electron_mass = 0.000548579909065

## Description of dataset records

`datasets` is the global dict that holds all data sets. It is a **dict of dict's**.

Each data set is **represented as a dict**.

Each record has the following fields (keys):

- `name`: the table/figure name of the data set
- `source`: the biological source for each dataset
- `mode`: the aquisition mode
- `alignment`: the alignment used to generate the data matrix
- `data`: the data matrix
- `target`: the sample labels, possibly already integer encoded
- `MDiN`: Mass-Difference Network - Not present here, only on sMDiNsAnalysis notebook
- `<treatment name>`: transformed data matrix / network. These treatment names can be
    - `Ionly`: missing value imputed data by 1/5 of the minimum value in each sample in the dataset, only
    - `NGP`: normalized, glog transformed and Pareto scaled
    - `Ionly_RF`: missing value imputed data by random forests, only
    - `NGP_RF`: normalized, glog transformed and Pareto scaled
    - `IDT`: `NGP_RF` or `NGP` - Intensity-based Data pre-Treatment chosen as comparison based on which of the two performed better for each dataset and each statistical method
    - `sMDiN`: Sample Mass-Difference Networks - Not present here, only on sMDiNsAnalysis notebook
       
- `<sMDiN analysis name>`: data matrix from nework analysis of MDiNs - Not in this notebook
    - `Degree`: degree analysis of each sMDiN
    - `Betweenness`: betweenness centrality analysis of each sMDiN
    - `Closeness`: closeness centrality of analysis of each sMDiN
    - `MDBI`: analysis on the impact of each MDB (Mass-Difference based building-block) on building each sMDiN
    - `GCD11`: Graphlet Correlation Distance of 11 different orbits (maximum of 4-node graphlets) between each sMDiN.
    - `WMDBI`: an alternative calculation of MDBI using the results from the degree analysis.

- `iter_fold_splits`: contains nested dicts that identify and contain each transformed training and testing groups data matrices with their respective iteration, training/test, fold number and one of the previously mentioned data pre-treatments
- `train`: specific to the HD dataset; contains a set of the different pre-treatments and sMDin analysis mentioned and a target based on the training set defined for HD
- `test`: specific to the HD dataset; contains a set of the different pre-treatments and sMDin analysis mentioned and a target based on the external test set defined for HD


The keys of `datasets` may be shared with dicts holding records resulting from comparison analysis.

Here are the keys (and respective names) of datasets used in this study:

- GD_neg_global2 (GDg2-)
- GD_neg_class2 (GDc2-)
- YD (YD)
- vitis_types (GD types)
- HD (HD)

### Description of grapevine data sets

Grapevine Dataset - 33 samples belonging to 11 different grapevine varieties (3 samples per variety/biological group) of FT-ICR-MS metabolomics data obtained in electrospray negative ionization mode.

5 different _Vitis_ species (other than _V. vinifera_) varieties:

- CAN - 3 Samples (14, 15, 16) of _V. candicans Engelmann_ (VIVC variety number: 13508)
- RIP - 3 Samples (17, 18, 19) of _V. riparia Michaux_ (Riparia Gloire de Montpellier, VIVC variety number: 4824) 
- ROT - 3 Samples (20, 21, 22) of _V. rotundifolia_ (Muscadinia Rotundifolia Michaux cv. Rotundifolia, VIVC variety number: 13586)
- RU - 3 Samples (35, 36, 37) of _V. rupestris Scheele_ (Rupestris du lot, VIVC variety number: 10389)
- LAB - 3 Samples (8, 9, 10) of _V. labrusca_ (Isabella, VIVC variety number: 5560)

6 different _V. vinifera_ cultivars varieties are:

- SYL - 3 samples (11, 12, 13) of the subspecies _sylvestris_ (VIVC variety number: -)
- CS - 3 Samples (29, 30, 31) of the subspecies _sativa_ cultivar Cabernet Sauvignon (VIVC variety number: 1929)
- PN - 3 Samples (23, 24, 25) of the subspecies _sativa_ cultivar Pinot Noir (VIVC variety number: 9279)
- REG - 3 Samples (38, 39, 40) of the subspecies _sativa_ cultivar Regent (VIVC variety number: 4572)
- RL - 3 Samples (26, 27, 28) of the subspecies _sativa_ cultivar Riesling Weiss (VIVC variety number: 10077)
- TRI - 3 Samples (32, 33, 34) of the subspecies _sativa_ cultivar Cabernet Sauvignon (VIVC variety number: 15685)

Data acquired by Maia et al. (2020):

- Maia M, Ferreira AEN, Nascimento R, et al. Integrating metabolomics and targeted gene expression to uncover potential biomarkers of fungal / oomycetes ‑ associated disease susceptibility in grapevine. Sci Rep. Published online 2020:1-15. doi:10.1038/s41598-020-72781-2
- Maia M, Figueiredo A, Silva MS, Ferreira A. Grapevine untargeted metabolomics to uncover potential biomarkers of fungal/oomycetes-associated diseases. 2020. doi:10.6084/m9.figshare.12357314.v1

**Peak Alignment** and **Peak Filtering** were performed with function `metabolinks.peak_alignment.align()`. Human leucine enkephalin (Sigma Aldrich) was used as the reference feature (internal standard, [M+H]+ = 556.276575 Da or [M-H]- = 554.262022 Da).

**2** data matrices were constructed from this data:

- `GD_neg_global2` (GDg2-) was generated after retaining only features that occur (globally) at least twice in all 33 samples of the dataset (filtering/alignment) for the **negative mode** data acquisition.
- `GD_neg_class2` (GDc2-) was generated after retaining only features that occur at least twice in the three replicates of at least one _Vitis_ variety in the dataset (filtering/alignment) for the **negative mode** data acquisition.

For the purpose of assessing the performance of supervised methods each of these four datasets was used with target labels defining classes corresponding to replicates of each of the 11 Vitis species/cultivars.

For the purpose of assessing the performance of supervised methods under a binary (two-class) problem, data set `GD_neg_class2` was also used with target labels defining two classes: Vitis vinifera cultivars and "wild", non-vinifera Vitis species. This is dataset `vitis_types` (GD types).

### Description of the yeast data set

Yeast dataset - 15 samples belonging to 5 different yeast strains of _Saccharomyces cerevisiae_ (3 biological replicates per strain/biological group) of FT-ICR-MS metabolomics data obtained in positive ionization mode. The 5 strains were: the reference strain BY4741 (represented as BY) and 4 single-gene deletion mutants of this strain – ΔGLO1 (GLO1), ΔGLO2 (GLO2), ΔGRE3 (GRE3) and ΔENO1 (ENO1). These deleted genes are directly or indirectly related to methylglyoxal metabolism.

Data acquired by Luz et al. (2021):

- Luz J, Pendão AS, Silva MS, Cordeiro C. FT-ICR-MS based untargeted metabolomics for the discrimination of yeast mutants. 2021. doi:10.6084/m9.figshare.15173559.v1

**Peak Alignment** and **Peak Filtering** was performed with MetaboScape 4.0 software (see reference above for details in sample preparation, pre-processing, formula assignment). In short, Yeast Dataset was obtained with Electrospray Ionization in Positive Mode and pre-processed by MetaboScape 4.0 (Bruker Daltonics). Human leucine enkephalin (Sigma Aldrich) was used as the reference feature (internal standard, [M+H]+ = 556.276575 Da or [M-H]- = 554.262022 Da).

A data matrix was constructed from this data: data set named **`YD` (YD)** was generated after retaining only features that occur (globally) at least twice in all 15 samples (filtering/alignment).

Furthermore, peaks higher than 1000 m/z were filtered out. Formula assignment was performed with MetaboScape 4.0 software and, to consider peaks that had the same formula assigned, these were joined together in a single peak (`feature_filter` function).

For the purpose of assessing the performance of supervised methods, this data set was used with target labels defining classes corresponding to replicates of each of the 4 yeast strains.

#### Loading yeast data set

In [ ]:
def read_MetScape_file(filename,
                          col_renamer=None,
                          add_labels=None,
                          remove_ref_feat=None,):
    
    """Read in a MetaboScape bucket table from a CSV file."""
    
    data = pd.read_csv(filename).set_index('Bucket label')
    
    # optionally rename sample_names
    if col_renamer is not None:
        data = data.rename(columns=renamer)
    
    # optionally remove a reference feature (if already normalized)
    if remove_ref_feat is not None:
        #print(f'Feature {remove_ref_feat}\n{data.loc[remove_ref_feat, :]}\n----------')
        data = data.drop(index=[remove_ref_feat])
        
    # Joining rows that have the same formula assigned (with feature_filter) while keeping rows without formulas assigned.
    data = feature_filter(data)
    
    # split in peak metadata and intensities
    peak_cols = ['m/z', 'Name', 'Formula']
    intensity_cols = [c for c in list(data.columns) if c not in peak_cols]
    peaks = data[peak_cols]
    intensities = data[intensity_cols]

    # replace zeros for NaN's
    intensities = intensities.replace(0, np.nan).dropna(how='all')
    
    # force peaks to have the same features as the (trimmed) intensities
    peaks = peaks.reindex(intensities.index)

    # optionally, add labels to intensities
    if add_labels is not None:
        intensities = mtl.add_labels(intensities, labels=add_labels)
    
    return {'filename': filename,
            'peaks':peaks,
            'intensities': intensities}

def renamer(colname):
    # Util to optionally remove all those 00000 from sample names
    return ''.join(colname.split('00000'))

In [ ]:
def feature_filter(Spectra):
    """Join all features that share the same formula by addition.
       
       Spectra: Pandas DataFrame (dataset).
       
       Returns: Pandas DataFrame with filtered dataset.
    """

    df = Spectra
    # Variables to store results
    rp = pd.DataFrame(columns = df.columns)
    rep = []
    # Series with number of times each formula appears in the dataset
    form_count = df['Formula'].value_counts()
    a = 0
    
    for i in df['Formula']:
        if pd.isnull(i) == True: # If no formula is assigned to the feature, the feature stays in the new dataset
            #rp = rp.append(df.iloc[a,:])
            rp.loc[df.iloc[a,:].name] = df.iloc[a,:]

        else: # If a formula is assigned
            if form_count[i] > 1: # If the formula appears more than one time in the dataset
                if i not in rep: # And If it didn't already appear before
                    
                    # Bucket label, name and formula info are the ones from the greater intensity m/z peak (of peaks with the
                    # same formula)
                    peaks = df.loc[df['Formula'] == i,:].iloc[:,3:].sum(axis = 1)
                    peaks_idx = peaks.idxmax()
                    info = df.loc[peaks_idx].iloc[:3] # Bucket label, name and formula info
                    
                    # Addition of all m/z peaks intensity with the same formula
                    newrow = df.loc[df['Formula'] == i,:].iloc[:,3:].sum(axis = 0)
                    newrow_c = pd.concat([info, newrow], axis = 0) # Join with the m/z peak metadata
                    # m/z peak is from the greater intensity m/z peak (of peaks with the same formula)
                    newrow_c = newrow_c.rename(df.loc[peaks_idx].name)
                    
                    # Append peak with intensity added from all peaks sharing the same formula
                    #rp = rp.append(newrow_c)
                    rp.loc[newrow_c.name] = newrow_c

                    rep.append(i) # Append repeating formulas so this process isn't repeated again for the other m/z peaks

            else:
                # Append peaks with formulas that aren't repeated in the dataset
                #rp = rp.append(df.iloc[a,:])
                rp.loc[df.iloc[a,:].name] = df.iloc[a,:]
                #print(df.iloc[a,:].name)

        a = a + 1

    # return DataFrame
    return rp

In [ ]:
# Labels of the 5 biological groups (5 yeast strains) - only added after
yeast_classes = 'WT ΔGRE3 ΔENO1 ΔGLO1 ΔGLO2'.split()

In [ ]:
# Read in the file and keep results in dicts 

prefix_to_drop = None # change to 'ENO' to remove ENO strain

# MetaboScape data
MS_data = read_MetScape_file('5yeasts_notnorm.csv', 
                             remove_ref_feat=None,
                             add_labels=None,
                             col_renamer=renamer)

In [ ]:
# keep features that appear in at least two samples
yeast_datamatrix = transf.keep_atleast(MS_data['intensities'].transpose(), minimum=2)

# Changing the index to be numbers - our list of neutral masses from the bucket lists.
new_columns = []
for i in range(len(yeast_datamatrix.columns)):
    new_columns.append(float(yeast_datamatrix.columns[i][:-3]))
yeast_datamatrix.columns = new_columns

# Only keep peaks below 1000 m/z - Very few peaks above 1000 m/z.
yeast_datamatrix = yeast_datamatrix.T[yeast_datamatrix.columns < 1000].T

yeast_datamatrix

In [ ]:
# keep features that appear in at least two samples
yeast_datamatrix = transf.keep_atleast(MS_data['intensities'].transpose(), minimum=2)

# Changing the index to be numbers - our list of neutral masses from the bucket lists.
new_columns = []
for i in range(len(yeast_datamatrix.columns)):
    new_columns.append(float(yeast_datamatrix.columns[i][:-3]))
yeast_datamatrix.columns = new_columns

# Only keep peaks below 1000 m/z - Very few peaks above 1000 m/z.
yeast_datamatrix = yeast_datamatrix.T[yeast_datamatrix.columns < 1000].T

yeast_datamatrix

### Description of the Human dataset

Human Dataset - 249 samples belonging to 2 different classes of HILIC-MS data obtained in positive ionization mode. The Chromatography system was a Thermo Dionex Ultimate 3000 with the column being a Waters Xbridge BEH HILIC (75 x 2.1mm, 2.5um). Mass Spectrometry was performed in a Thermo Q Exactive HF hybrid Orbitrap operating in positive electrospray ionization mode. Further information in the data deposition site mentioned below.

The 2 classes are:

- 135 pre-operative blood (serum) samples from patients with **'No Recurrence'** of Prostate Cancer after Radical Prostatectomy
- 114 pre-operative blood (serum) samples from patients with **'Recurrence'** of Prostate Cancer after Radical Prostatectomy

I think this means samples were collected before the Prostatectomy before knowledge of recurrence or no recurrence? There was a total of 80 subjects.

Data taken from data deposition site: https://www.metabolomicsworkbench.org/data/DRCCMetadata.php?Mode=Study&DataMode=AllData&StudyID=ST001082&StudyType=MS&ResultType=5#DataTabs

This data is available at the NIH Common Fund's National Metabolomics Data Repository (NMDR) website, the Metabolomics Workbench, https://www.metabolomicsworkbench.org where it has been assigned Project ID PR000724. The data can be accessed directly via it's Project DOI: 10.21228/M83D5V. 

Data deposited by the Georgia Institute of Technology and researcher Facundo Fernandez. This data was taken already aligned in a 2D numerical data matrix. 

Data used in the paper: Clendinen CS, Gaul DA, Monge ME, et al. Preoperative Metabolic Signatures of Prostate Cancer Recurrence Following Radical  Prostatectomy. J Proteome Res. 2019;18(3):1316-1327. doi:10.1021/acs.jproteome.8b00926

Dataset named **HD (HD)** was generated after retaining only features that occur (globally) at least twice in all samples of the dataset. For the purpose of assessing the performance of supervised methods this dataset was used with target labels defining classes based on which of the 2 classes mentioned prior, the samples belonged to.


The data read from the site seemed to be completely untreated, which was perfect for our efforts. We can't be sure but the presence of plenty of missing values (very useful for us) points to no great peak filtering and missing value imputation being made, the bar plot built from adding up all intensities of each sample seems to point that no normalization had yet been employed in the data, and the distribution observed in the boxplot of the intensities of some of the features in the samples seems to point that no scaling had been used yet in the data as well.

The data also had 2 copies of each sample, where the 2nd copies were equal to the first ones multiplied by a constant. This constant was different for each sample. We removed the presence of the 2nd copies. The index (metabolites) were reduced from the notation m/z value_retention time to only have the m/z value.


#### Loading Human dataset

In [ ]:
# Reading the dataset, this dataset has two copies of the samples.The 2nd copy is equal to the 1st multiplied by a constant
# unique to each sample.
human_datamatrix_base = pd.read_excel('ST001082_AN001766_HD.xlsx')

In [ ]:
human_datamatrix = human_datamatrix_base.iloc[:-1, :-4] # Just select the rows corresponding to the dataset
mz_list = human_datamatrix_base.iloc[:-1, -4] # Select column with list of m/z values

human_datamatrix = human_datamatrix.set_index(human_datamatrix.columns[0]) # Set index as the metabolites name
human_datamatrix = human_datamatrix.replace({0:np.nan}) # Replacing 0 values as missing values

human_datamatrix

In [ ]:
# Repeating m/z values with different retention times

#abc = []
#for i in list(mz_list):
#    if i in abc:
#        print(i)
#    else:
#        abc.append(i)
#len(set(list(mz_list)))

In [ ]:
# Select one of the two copies of samples in the dataset by removing the samples ending with '.1'
human_datamatrix = human_datamatrix[[i for i in human_datamatrix.columns if not i.endswith('.1')]]

human_datamatrix = human_datamatrix.T # Transpose the dataset

In [ ]:
# Current sample labels. Sample will consist of 'Sample Type:No Recurrence' and 'Sample Type:Recurrence'
hd_labels = list(human_datamatrix['Factors'])
set(hd_labels)

In [ ]:
# How many 'No Recurrence' samples in the dataset
hd_labels.count('Sample Type:No Recurrence')

### Blank Treatment

- If blank_treatment = True:

An average of the blanks will be subtracted to the remaining dataset. Missing values in the blanks were replaced by 0 to calculate the average of the blanks. Negative values that arise in the dataset from subtracting the blanks will be coded as 0/missing values in the dataset.

- If blank_treatment = False:

Blank samples are removed and not accounted for. 

In [ ]:
blank_treatment = True
#blank_treatment = False

In [ ]:
if blank_treatment:
    blanks = human_datamatrix[human_datamatrix['Factors'] == 'Sample Type:Blank'].iloc[:,1:] # Select blank samples
    blanks = blanks.replace({np.nan:0}) 
    blanks = blanks.astype(float) # Get blank samples with floats (needed since datamatrix has strings)
    blanks_average = blanks.mean() # Average of the blanks
    blanks_average

In [ ]:
# Selecting the samples belonging to either the 'No Recurrence' or 'Recurrence' 
selection = []
for i in human_datamatrix.loc[:, 'Factors']:
    if i in ['Sample Type:No Recurrence', 'Sample Type:Recurrence']:
        selection.append(True)
    else:
        selection.append(False)

In [ ]:
human_datamatrix = human_datamatrix[selection]
# Creating the list of 'targets' (labels of samples) of the dataset with the 'No Recurrence' and 'Recurrence' classes 
hd_labels = []
for i in list(human_datamatrix.iloc[:,0]):
    if i == 'Sample Type:No Recurrence':
        hd_labels.append('No Recurrence')
    else:
        hd_labels.append('Recurrence')

In [ ]:
human_datamatrix = human_datamatrix.iloc[:,1:]
human_datamatrix

In [ ]:
human_datamatrix = human_datamatrix.astype(float) # Passing the values from strings to floats.

In [ ]:
if blank_treatment:
    human_datamatrix = human_datamatrix.replace({np.nan:0}) - blanks_average
    human_datamatrix[human_datamatrix<0] = 0
    human_datamatrix = human_datamatrix.replace({0:np.nan})

In [ ]:
#human_datamatrix.columns = list(mz_list)[1:]
#human_datamatrix

Set list of masses to pass to Cytoscape and to build the MDiNs.

'Neutralization' of the _m/z_ values by subtracting the mass of a proton to the _m/z_ peaks.

4 peaks (2 pairs) with different retention time have the exact same _m/z_ values. These 2 pairs of peaks wil be treated as only 2 different peaks (instead of four) for MDiN building purposes.

In [ ]:
mz_list = mz_list[1:] - chemdict['H'][0] + electron_mass
counts = human_datamatrix.count(axis=0)
final_mz_list = list(mz_list[list(counts >= 2)])
final_mz_list = set(final_mz_list)

In [ ]:
human_datamatrix = transf.keep_atleast(human_datamatrix, minimum=2) # Keep features that appear in at least two samples
human_datamatrix

### Building datasets database

Atomic mass of the most common naturally occurring isotope of the more important elements in metabolites and their isotopic abundances.

In [ ]:
datasets = {}
# From the alignments_new store

# GD_neg_global2 (GDg2-)
data_df = pd.HDFStore('GD_data.h5').get('all_1ppm_min2_neg').transpose()
gd_labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
new_cols_df = data_df.columns + chemdict['H'][0] - electron_mass
data_df.columns = new_cols_df

datasets['GD_neg_global2'] = {'source': 'grapevine',
                              'alignment': '1-2',
                              'mode': '-',
                              'name': 'GDg2-',
                              'data': data_df,#.to_dict(orient='split'),
                              'original': data_df,#.to_dict(orient='split'),
                              'target': gd_labels,
                              'classes': list(pd.unique(gd_labels))}

# GD_neg_class2 (GDc2-)
data_df = pd.HDFStore('GD_data.h5').get('groups_1ppm_min2_all_1ppm_neg').transpose()
labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
new_cols_df = data_df.columns + chemdict['H'][0] - electron_mass
data_df.columns = new_cols_df

datasets['GD_neg_class2'] = {'source': 'grapevine',
                              'alignment': '2-1',
                              'mode': '-',
                              'name': 'GDc2-',
                              'data': data_df,#.to_dict(orient='split'),
                              'original': data_df,#.to_dict(orient='split'),
                              'target': gd_labels,
                              'classes': list(pd.unique(gd_labels))}

# YD (YD)
yeast_labels = [item for item in yeast_classes for i in range(3)]

datasets['YD'] = {'source': 'yeast',
                            'alignment': '1-2',
                            'mode': '+',
                            'name': 'YD',
                            'data': yeast_datamatrix,#.to_dict(orient='split'),
                            'original': yeast_datamatrix,#.to_dict(orient='split'),
                            'target': yeast_labels,
                            'classes': list(pd.unique(yeast_labels))}

# vitis_types (GD types)
vitis_types = {'CAN': 'wild', 'RIP': 'wild', 'ROT': 'wild','RU': 'wild', 'LAB': 'wild',
               'SYL': 'wild','REG': 'vinifera','CS': 'vinifera','PN': 'vinifera','RL': 'vinifera',
               'TRI': 'vinifera'}

gd_type_labels = [vitis_types[lbl] for lbl in gd_labels]

datasets['vitis_types'] = {'source': 'grapevine',
                            'alignment': '2-1',
                            'mode': '-',
                            'name': 'GD types',
                            'data': datasets['GD_neg_class2']['original'].copy(),
                            'original': datasets['GD_neg_class2']['original'].copy(),
                            'target': gd_type_labels,
                            'classes': list(pd.unique(gd_type_labels))}

# HD (HD)
datasets['HD'] = {'source': 'human',
                  'alignment': '1-2',
                  'mode': '+',
                  'name': 'HD',
                  'data': human_datamatrix,#.to_dict(orient='split'),
                  'original': human_datamatrix,#.to_dict(orient='split'),
                  'target': hd_labels,
                  'classes': list(pd.unique(hd_labels))}


print('target for grapevine 11-variety data sets')
print(datasets['GD_neg_global2']['target'])
print('------\ntarget for 4 yeast strains data set')
print(datasets['YD']['target'])
print('------\ntarget for 2-class wild Vitis vs Vitis vinifera data set')
print(datasets['vitis_types']['target'])
print('------\ntarget for human 2-class dataset')
print(datasets['HD']['target'])

In [ ]:
#data_df = pd.HDFStore('alignments_new.h5').get('groups_1ppm_min2_all_1ppm_neg').transpose()
#labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
#new_cols_df = data_df.columns + chemdict['H'][0] - electron_mass
#data_df.columns = new_cols_df

#df = datasets['GD_neg_global2']['data']

In [ ]:
# should not raise AssertionError:
#assert_frame_equal(df, data_df1)

### Writing a .csv file for each of the datasets with the list of masses that will be used to build the MDiNs (in Cytoscape's MetaNetter)

The file needs to be correctly read by Cytoscape's MetaNetter (and Cytoscape) where the network will be built.

For this, files with two columns both with the same list of masses (one index, one in the column) will be made. One will correspond to the network nodes names as a 'string'. The other will be an attribute of the corresponding node as a 'float' and will be used in the building process.

In [ ]:
for dskey, ds in datasets.items():
        
    if dskey.startswith('HD'):  
        # Specific mass lists for HD 
        # (two pairs of masses with same m/z values and different retention times are only treated as 2 different m/z)
        final_mz_list = set(final_mz_list)
        pd.DataFrame(final_mz_list, index=final_mz_list).to_csv('mass_data/MassList' + dskey + '.csv')
        continue
    
    # Creating mass lists in .csv files for MDiN building
    pd.DataFrame(ds['data'].columns, index=ds['data'].columns).to_csv('mass_data/MassList' + dskey + '.csv')

In [ ]:
datasets.keys()

### Dataset Characteristics

Building a table with general characteristics about the datasets studied.

In [ ]:
def characterize_dataset(dskey, ds):
    "Computes and returns some general characteristics of a dataset in a dictionary."

    dataset_chrs = {}
    
    name = ds['name'] # Name of the dataset
    n_samples, n_feats = ds['data'].shape
    n_classes = len(ds['classes'])
       
    Feat_Sample = ds['data'].count(axis=1) # Nº Features in each sample
    Min_Feat_Sample = str(Feat_Sample.min()) # Minimum nº Features in a sample
    Max_Feat_Sample = str(Feat_Sample.max()) # Maximum nº Features in a sample
    Average_Feat_Sample = Feat_Sample.mean() # Average nº Features in a sample
    
    avg_feat_per_sample = int(round(Average_Feat_Sample,0)) # Round
    
    Samp_Class = len(ds['target'])/len(ds['classes']) # Nº Sample per Class
    if dskey == 'vitis_types':
        Samp_Class = '15 Vitis vinifera, 18 Wild'
        #Samp_Class = '15 $\it{Vitis}$ $\it{Vinifera}$, 18 Wild'
    elif dskey == 'HD':
        Samp_Class = '114 Recurrence, 135 no Recurrence'
    else:
        Samp_Class = str(int(Samp_Class))
    
    n_na = ds['data'].isna().sum().sum() # Nº of missing values in the dataset
    
    p_na = round(100.0 * n_na / (n_samples * n_feats), 2) # % of missing values in the dataset
    
    avg_na_per_feature = (n_samples - ds['data'].count(axis=0)).mean()
    avg_na_per_feature = int(round(avg_na_per_feature, 0))
    
    return {'Data set': name,
            '# samples': n_samples,
            '# features': n_feats,
            'features / sample (range)': f'{avg_feat_per_sample} ({Min_Feat_Sample}-{Max_Feat_Sample})',
            '# classes': n_classes,
            'samples / class':Samp_Class,
            '% missing values': p_na,} 
            #'missing values / feature': avg_na_per_feature}

data_characteristics = [characterize_dataset(dskey, ds) for dskey, ds in datasets.items()]
data_characteristics = pd.DataFrame(data_characteristics).set_index('Data set')
data_characteristics

In [ ]:
data_characteristics.to_excel('dataset_characteristics.xlsx', index=True)

## Sample Mass-Difference Network Construction

#### MDBs (Mass-Difference-based Building blocks) 

Building the **list of MDBs** to use when building the MDiNs.

MDB - Mass-Difference-based Building blocks.

The choice of this MDBs is then crucial in the network building process. Ideally, they should represent different 'simple' biochemical reactions that cover the most common and ubiquitous enzymatic and non-enzymatic reactions while also being a relatively small amount of reactions – a total of 15 were picked - and maintaining the metabolite formula charge neutrality. 

For example, to maintain neutrality, a phosphorylation would mean the overall addition of a PO3H – addition of a -PO3$^{2-}$ group + 2 H$^+$ (maintaining neutrality) to replace an H atom in a metabolite. All the MDBs chosen represent changes in metabolites of no more than 5 atoms and less than 80 Da (small size). Each MDB should represent a set of chemically known reactions and a change in every main element in metabolites (C, H, O, N, S and P) is represented by at least one of the MDBs. To fulfil these conditions, representative MDBs were searched using BRENDA (https://www.brenda-enzymes.org/). The groups chosen were the following:

- CH2 (methylation) 
- O (oxygenation) 
- H2 (Hydrogenation)
- O(N-H-) (Aminase): NH3(O-) - H2
- PO3H (phosphorylation)
- NH3(O-) (transaminases)
- SO3 (sulphation)
- CO (like formylation) 
- CO2 (carboxylation, decarboxylation)
- CHOH (Hydroxymethylation) 
- NCH (formidoyltransferase)
- CONH (carbamoyltransferase)
- C2H2O (acetylation)
- S (rare but an extra S reaction)
- H2O

For the YD dataset, an extra 2 MDBs were used as an example of specific MDBs that can be used for certain datasets. These two MDBs were added since these reactions are known to occur with higher glycation in a biological system. Despite mainly targeting proteins, some polyssaccarides can also be glycated. Since the yeast mutants gene deletion affect enzymes involved in methylglyoxal metabolism (ΔGLO1, ΔGLO2, ΔGRE3), which is an anti-glycation mechanism in the cell.

- CHCOOH (carboxymethylation)
- CCH3COOH (carboxyethylation)

There could be many other MDB representing other reactions that can also be included such as CN2H2 (amidinotransferases), COCH2COO (malonyl transferases), etc.

Functions to transform formulas in string format to dictionary format and from calculate formulas (in dictionary format) exact masses.

Example:

    'String' format: 'C6H1206'
    
    'Dictionary' format: {'C':6, 'H':12, 'O':6}

In [ ]:
def formula_process(formula):
    """Transforms a formula in string format into a dictionary."""

    # Empty dictionary to store the results
    results = dict.fromkeys(['C','H','O','N','S','P','Cl','F'], 0)
    count = ''
    letter = None
    minus = False
    
    # Run through the string
    for i in range(len(formula)):
        if formula[i].isupper(): # If i is an uppercase letter then it is an element
            if letter: # Just to account for the first letter in the formula where letter is None 
                # Reached another letter, store previous results and reset count
                if minus:
                    results[letter] = - int(count or 1)
                    minus = False
                else:
                    results[letter] = int(count or 1)
                count = ''
                
            if i+1 < len(formula): # In case it's a two letter element such as Cl
                if formula[i+1].islower(): # The second letter is always lower case
                    letter = formula[i] + formula[i+1] # Store new 2 letter element
                    continue
                    
            letter = formula[i] # Store new 1 letter element
            
        elif formula[i].isdigit():
            count = count + formula[i] # If number, add number to count
        
        elif formula[i] == '-':
            minus = True
    
    # Store results of the last letter
    if minus:
        results[letter] = - int(count or 1)
        minus = False
    else:
        results[letter] = int(count or 1)
                    
    return results

In [ ]:
def getmass(c,h,o,n,s,p,cl,f):
    "Get the exact monoisotopic mass for any formula with the C,H,O,N,S,P,Cl and F elements only."
    massC = chemdict['C'][0] * c
    massH = chemdict['H'][0] * h
    massO = chemdict['O'][0] * o
    massN = chemdict['N'][0] * n
    massS = chemdict['S'][0] * s
    massP = chemdict['P'][0] * p
    massCl = chemdict['Cl'][0] * cl
    massF = chemdict['F'][0] * f 

    massTotal = massC + massH + massO + massN + massS + massP + massCl + massF

    return massTotal

#### Making the transformation file with MDBs to use in Cytoscape

The transformation file will be in a format accepted by Cytoscape's MetaNetter (.txt separated by tabs) to be considered as a list of transformations (with specific masses) to make the MDiN. 4 columns: 'Label', 'Formula', 'Mass', 'Selected' with mass being the most important.

Formula changes will be put under the 'Label' column and the corresponding set of reactions they represent will be put under 'Formula'. This switch is to make the MDB Impact analysis done later down the line more legible (since 'formula changes' are easier to observe, quickly understood and more concise in relation to long-winded names for set of reactions).

The 'Selected' column will be just a set of True for the MDBs to come in pre-selected in Cytoscape.

In [ ]:
# Prepare DataFrame
trans_groups = pd.DataFrame(columns=['Label','Formula','Mass','Selected'])
trans_groups = trans_groups.set_index('Label')

# Chemical Formula transformations (MDBs chosen)
MDBs = ['H2','CH2','CO2','O','CHOH','NCH','O(N-H-)','S','CONH','PO3H','NH3(O-)','SO3','CO', 'C2H2O', 'H2O']
# Examples of the set of Reactions each MDB represents
labels = ['Hydrogenation','Methylation','Carboxylation/Decarboxylation','Oxygenation','Hydroxymethylation',
          'Formidoylation','Deamination','Tranfer of and -SH group','Carbamoylation','Phosphorylation','Transamination',
          'Sulphation','Formylation','Acetylation','Condensation/Dehydration/Cyclization']

for i in range(len(MDBs)): # Passing through each transformation/MDB

    # Calculate the exact mass of the MDBs
    form = formula_process(MDBs[i])
    MDB_mass = getmass(form['C'],form['H'],form['O'],form['N'],form['S'],form['P'],form['Cl'],form['F'])
        
    # Making the row for each transformation
    string = MDBs[i]
    trans_groups.loc[string] = (labels[i], MDB_mass, 'true')

trans_groups = trans_groups.sort_values(by='Mass')
trans_groups # Transformations DataFrame

In [ ]:
#Saving the df as a .txt file in the described format
trans_groups.to_csv('transgroups.txt', header=False, sep='\t')

MDB Transformation file for the YD dataset (that includes the aforementioned two extra MDBs)

In [ ]:
# Prepare DataFrame
trans_groups = pd.DataFrame(columns=['Label','Formula','Mass','Selected'])
trans_groups = trans_groups.set_index('Label')

# Chemical Formula transformations (MDBs chosen)
MDBs = ['H2','CH2','CO2','O','CHOH','NCH','O(N-H-)','S','CONH','PO3H','NH3(O-)','SO3','CO', 'C2H2O', 'H2O', 
        'C2H2O2', 'C3H4O2']
# Examples of the set of Reactions each MDB represents
labels = ['Hydrogenation','Methylation','Carboxylation/Decarboxylation','Oxygenation','Hydroxymethylation',
          'Formidoylation','Deamination','Tranfer of and -SH group','Carbamoylation','Phosphorylation','Transamination',
          'Sulphation','Formylation','Acetylation','Condensation/Dehydration/Cyclization', 'Carboxymethylation',
         'Carboxyehtylation']

for i in range(len(MDBs)): # Passing through each transformation/MDB

    # Calculate the exact mass of the MDBs
    form = formula_process(MDBs[i])
    MDB_mass = getmass(form['C'],form['H'],form['O'],form['N'],form['S'],form['P'],form['Cl'],form['F'])
        
    # Making the row for each transformation
    string = MDBs[i]
    trans_groups.loc[string] = (labels[i], MDB_mass, 'true')

trans_groups = trans_groups.sort_values(by='Mass')
trans_groups # Transformations DataFrame

In [ ]:
#Saving the df as a .txt file in the described format
trans_groups.to_csv('transgroups_YD.txt', header=False, sep='\t')

### Build the Mass-Difference Networks in Cytoscape using MetaNetter 2.0

To build the network in Cytoscape we'll need:

1) A list of neutral masses (with masses as float in node attributes).

2) A list of allowed transformations with specific mass differences - list of MDBs that we build above.

The transformation files and the dataset files were used in Cytoscape to build the MDiNs.

Parameters: 1 ppm error allowed for edge establishment.

The Yeast datasets already have the m/z 'buckets' that are already representing the neutral masses of the metabolites. To consider the neutral masses of the grapevine datasets, a simple transformation by adding or removing a proton (Hydrogen atom mass - electron mass), depending if the ionization mode is negative or positive, respectively, on the m/z peaks was made.


Only one full network (for each benchmark dataset) is built. Then, subgraphs of them will be used to select every sample MDiN. This is the same as building an MDiN for each sample.

The networks were exported in graphml format that networkX module can read.

- Nodes have a standard number ID instead of the mass (stored as the attribute 'mass'). Other attributes stored are irrelevant. 
- Edges among the different attributes have a very useful attribute called 'Transformation' which stores which MDB of the list was used to establish the edge - will be used for MDB Impact analysis and Weighted MDB Impact.
- Finally, the graph is directed. Since reactions are bidireccional, they will be transformed to undirected graphs.

Changes that will be made to the network:

- Nodes will be identified by their masses.
- Intensities of the node in each sample will be given in other notebooks to store for each specific subgraph.

## Data transformations and pre-treatments

Each dataset is transformed by 2 combinations of missing value imputation and data pre-treatment methods (generating datasets with different treatments applied).

### Traditional and more established  intensity-based Pre-Treatments

Missing value imputation is mandatory for the datasets since many statistical methods performed downstream can't handle missing values. 

Two missing value imputation procedures were applied before any intensity-based pre-treatments: 

- Missing values of a sample were replaced with a fifth of the minimum intensity value present in that sample of the data matrix - Limit of Detection type of imputation. This was performed with the `fillna_frac_sample_min` function below. 

- Missing values were imputed by Random Forest missing value imputation. Parameters: 10 trees, 100 nearest features considered, 0 minimum value, 1.0e10 maximum value and 5 max iterations for all datasets except HD (2 max iterations), other parameters were left as default. This was performed by using the scikit-learn Python module with `IterativeImputer`and `ExtraTreesRegressor` as estimator. In cases where this method was used, an extra **`_RF`** was added to the 'name' of the pre-treatment.

#### Combinations of intensity-based pre-treatments made:

- Ionly Treatment - Only Missing Value Imputation.

- **NGP Treatment** - Missing Value Imputation, Normalization, Generalized Logarithmic Transformation and Pareto Scaling.

For normalizations, Normalization by reference feature (Leucine Enkephalin) is used in all cases, except for the HD dataset where Probabilistic Quotient Normalization (PQN) is used with the mean of all samples acting as the reference sample.

Note: Leucine Enkephalin peak (reference feature) is removed upon normalization. Order of pre-treatments is the order in which they were mentioned.

In [ ]:
def impute_RF(df, nearest_features=100, n_trees=50, max_iter=10):
    "Random Forest Imputation of Missing Values."
    rf_estimator = ExtraTreesRegressor(n_estimators=n_trees)
    imputer = IterativeImputer(random_state=0, estimator=rf_estimator,
                           n_nearest_features=nearest_features,
                           min_value=0.0, max_value=1.0e10,
                           max_iter=max_iter,
                           verbose=0)
    imputed_data = imputer.fit_transform(df)
    res = pd.DataFrame(imputed_data, index=df.index, columns=df.columns)
    ncols = len(res.columns)
    res = res.dropna(axis='columns', how='any')
    ncols2 = len(res.columns)
    if ncols > ncols2:
        print(f'{ncols-ncols2} features dropped')
    return res

In [ ]:
# For missing value imputation based on constants relative to the minimum of each feature instead of the full dataset
def fillna_frac_feat_min(df, fraction=0.2):
    """Set NaN to a fraction of the minimum value in each column of the DataFrame."""

    minimum = df.min(axis=0) * fraction
    return df.fillna(minimum)

# For missing value imputation based on constants relative to the minimum of each sample instead of the full dataset
def fillna_frac_sample_min(df, fraction=0.2):
    """Set NaN to a fraction of the minimum value in each row of the DataFrame."""

    minimum = df.min(axis=1) * fraction
    return df.T.fillna(minimum).T

In [ ]:
# Performs all pre-treatment combinations mentioned
def compute_transf(dataset, norm_ref=None, lamb=None, max_iter=10):
    "Computes pre-treatments and missing value imputations and returns after treatment datasets in a dict."
    
    data = dataset['data']
    
    # Imputation of Missing Values
    #imputed = fillna_frac_feat_min(data, fraction=0.2)
    imputed = fillna_frac_sample_min(data, fraction=0.2)
    
    # Imputation by RF
    datacols = data.columns
    data.columns = [str(col) for col in data.columns]
    imputedRF = impute_RF(data, nearest_features=100, n_trees=10, max_iter=max_iter)
    imputedRF.columns = datacols
    imputed.columns = datacols
    
    # Normalization by a reference feature
    if norm_ref is not None:
        norm = transf.normalize_ref_feature(imputed, norm_ref, remove=True)
    else:
        norm = transf.normalize_PQN(imputed, ref_sample='mean')
    
    # Normalization by a reference feature RF
    if norm_ref is not None:
        normRF = transf.normalize_ref_feature(imputedRF, norm_ref, remove=True)
    else:
        normRF = transf.normalize_PQN(imputedRF, ref_sample='mean')
    
    # Normalization by PQN for HD - 1/2 and RF   
    #if dataset['name'].startswith('HD'):
      #  norm = transf.normalize_PQN(imputed, ref_sample='mean')
      #  normRF = transf.normalize_PQN(imputedRF, ref_sample='mean')

    # Pareto Scaling and Generalized Logarithmic Transformation
    NGP = transf.pareto_scale(transf.glog(norm, lamb=lamb))

    # Pareto Scaling and Generalized Logarithmic Transformation
    NGP_RF = transf.pareto_scale(transf.glog(normRF, lamb=lamb))
    
    # Store results
    dataset['Ionly'] = imputed
    dataset['NGP'] = NGP
    
    dataset['Ionly_RF'] = imputedRF
    dataset['NGP_RF'] = NGP_RF    

Human leucine enkephalin (Sigma Aldrich) is the reference feature (internal standard, [M+H]+ = 556.276575 Da or [M-H]- = 554.262022 Da) used for these datasets.

Search in the grapevine data sets for the reference feature and confirm the reference feature in the yeast data sets with `find_closest_features` from metabolinks package.

There is **no** reference feature in the HD dataset.

In [ ]:
# Theoretical mass for negative mode Leucine Enkephalin - 554.262022
# Theoretical mass for positive mode Leucine Enkephalin - 556.276575
Leu_Enk_neg = 554.262022 + chemdict['H'][0] - electron_mass
Leu_Enk_pos = 556.276575 - chemdict['H'][0] + electron_mass

# Reference Feature in the yeast dataset
leu_enk_name = 555.2692975341

In [ ]:
query_datasets = [name for name in datasets if datasets[name]['source']=='grapevine']

for name in query_datasets:
    ds = datasets[name]
    print(f'looking for reference in {name} ...')
    ref_variable = Leu_Enk_neg if ds['mode'] == '-' else Leu_Enk_pos
    closest = transf.find_closest_features(ds['data'], features=[ref_variable])
    if closest[ref_variable] is not None:
        print('Found ref feature', ref_variable)
        delta = closest[ref_variable] - ref_variable
        print(f'In data: {closest[ref_variable]} delta = {delta:.3e}\n')

query_datasets = [name for name in datasets if name.startswith('YD')]

ref_variable = leu_enk_name
for name in query_datasets:
    ds = datasets[name]
    print(f'looking for reference in {name} ...') 
    closest = transf.find_closest_features(ds['data'], features=[ref_variable])
    if closest[ref_variable] is not None:
        print('Found ref feature', ref_variable, '\n')
    else:
        print('Ref feature not found\n')

##### Apply the different pre-treatments and get the results in their respective dictionaries

In [ ]:
for name, ds in datasets.items():
    dataset_name = datasets[name]["name"]
    print(f'Applying pre-processing transformations to data in {dataset_name}', end=' ...')
    start = perf_counter()
    
    max_iter = 5
    
    if name.startswith('YD'):
        ref_variable = leu_enk_name
    elif name.startswith('HD'):
        ref_variable = None
        max_iter = 2
    else:
        ref_variable = Leu_Enk_neg if ds['mode'] == '-' else Leu_Enk_pos
    
    compute_transf(ds, norm_ref=ref_variable, max_iter=max_iter)
    
    end = perf_counter()

    print(f'done! took {(end - start):.3f} s')

### Splitting data into test and train groups and performing independent data pre-treatments

For supervised analysis, where stratified k-fold cross-validation is used to assess model performance over 20 iterations (to consider different combinations of fold splits), the separation of the datasets into the train and testing folds for each iteration is generated.

This is generated now so we can perform the previously mentioned data pre-treatment independently to the training and testing set for each iteration and each 'fold' in order to avoid the data leakage that may happen to the established intensity based data pre-treatment.

For the features in the training sets that do not appear in test sets; these are added to the test data with the lowest value in the training set for that feature being assigned to all samples of the test set.

For the set of pre-treatments that use Random Forest Imputation, since you need at least 2 values in each feature to perform imputation, training sets were filtered so as to each features is at least present in 2 samples. In test sets, RF Imputation is performed on features that are present in at least 2 samples, but since our data has a very limited number of samples and each test data only has 1 sample of each class, we cannot eliminate features that are only present in one of the samples of the test group since that feature might be characteristic and act as a biomarker of the class that sample belongs to. To counteract that, 1/5 min imputation (a fifth of the minimum intensity value present in that sample of the data matrix) is performed on those features specifically to keep their potential information in the test data. Only features that also appear in the training data are kept for the test data.

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [ ]:
# Altered version of compute_transf to take into account training and testing sets
def compute_transf_iterations(dataset, norm_ref=None, lamb=None, train=True, max_iter=10):
    "Computes combinations of pre-treatments and returns after treatment datasets in a dict."
    
    data = dataset['data'].copy()
    data = data.dropna(how='all', axis=1)
    
    # Imputation of Missing Values
    imputed = fillna_frac_sample_min(data, fraction=0.2)
    
    # Imputation by RF
    # If training set, keep only features that appear at least in 2 samples
    if train:
        data_temp = transf.keep_atleast(data, minimum=2)
    # If test set, separate data between features that appear in at least 2 samples and those that only appear in 1.
    # Treat the data that only appears in 1 sample with with 1/5 min (of the sample) imputation.
    # Treat the rest of the data normally with RF Imputation as with the training set.
    # Then re-join the two sets of data
    else:
        data_temp = transf.keep_atleast(data, minimum=2).copy()
        remaining_data = data[np.setdiff1d(data.columns,data_temp.columns)]
        
        # Replacing missing values in remaining data with 1/5 of the minimum value in each sample
        fraction = 0.2
        minimum = data.min(axis=1) * fraction # Minimum value in each sample
        remaining_data = remaining_data.T.fillna(minimum).T

    datacols = data_temp.columns
    data_temp.columns = [str(col) for col in data_temp.columns]
    #print(data)
    imputedRF = impute_RF(data_temp, nearest_features=100, n_trees=10, max_iter=max_iter)
    imputedRF.columns = datacols
    #imputed.columns = datacols
    
    # Join the split data
    if not train:
        imputedRF = pd.concat((imputedRF, remaining_data), axis=1)
    
    # Normalization by a reference feature
    if norm_ref is not None:
        norm = transf.normalize_ref_feature(imputed, norm_ref, remove=True)
    else:
        norm = transf.normalize_PQN(imputed, ref_sample='mean')
    
    # Normalization by a reference feature RF
    if norm_ref is not None:
        normRF = transf.normalize_ref_feature(imputedRF, norm_ref, remove=True)
    else:
        normRF = transf.normalize_PQN(imputedRF, ref_sample='mean')
    
    # Pareto Scaling and Generalized Logarithmic Transformation
    NGP = transf.pareto_scale(transf.glog(norm, lamb=lamb))

    # Pareto Scaling and Generalized Logarithmic Transformation
    NGP_RF = transf.pareto_scale(transf.glog(normRF, lamb=lamb))
    
    # Store results
    dataset['Ionly'] = imputed
    dataset['NGP'] = NGP
    
    dataset['Ionly_RF'] = imputedRF
    dataset['NGP_RF'] = NGP_RF    

In [ ]:
# Number of iterations that will be used for cross-validation
iter_num = 20

for name, ds in datasets.items():
    
    # Save the HD for later
    if name == 'HD':
        continue
    
    np.random.seed(16)
    
    start2 = perf_counter()
    # Set up the place where data will be stored
    ds['iter_fold_splits'] = {}
    
    # Set up ref_variable for normalization and iteration number for imputation by RF
    max_iter = 2
    
    if name.startswith('YD'):
        ref_variable = str(leu_enk_name)
    elif name.startswith('HD'):
        ref_variable = None
    else:
        ref_variable = '555.269296452291' #if ds['mode'] == '-' else '555.269293547709'
    
    # In each iteration
    for n_iter in range(iter_num):
        n_fold = 5 if name in ('vitis_types', 'HD') else 3
        kf = StratifiedKFold(n_fold, shuffle=True, random_state=307*(n_iter+1))
        
        # Set up different storages
        ds['iter_fold_splits'][n_iter+1] = {}
        test_dict = ds['iter_fold_splits'][n_iter+1]
        test_dict['train'] = {} # For training set
        test_dict['test'] = {} # For test set
        fold = 1
        start = perf_counter()
        
        # Generate each fold of training and test sets for each iteration, treat them and store them
        for train_index, test_index in kf.split(ds['data'], ds['target']):
            
            # Generating
            test_dict['train'][fold] = {}
            test_dict['test'][fold] = {}
            test_dict['train'][fold]['data'], test_dict['test'][fold]['data'] = ds['data'].iloc[
                train_index, :], ds['data'].iloc[test_index, :]
            test_dict['train'][fold]['target'], test_dict['test'][fold]['target'] = [
                ds['target'][i] for i in train_index], [ds['target'][i] for i in test_index]
            
            # Treating
            compute_transf_iterations(test_dict['train'][fold], norm_ref=ref_variable, train=True, max_iter=max_iter)
            compute_transf_iterations(test_dict['test'][fold], norm_ref=ref_variable, train=False, max_iter=max_iter)
            
            # Adding features that do not appear in the test data but appear in the training data to the test data
            # with the minimum value of said feature in the training data 
            train_cols = test_dict['train'][fold]['data'].dropna(how='all', axis=1).columns
            test_cols = test_dict['test'][fold]['data'].dropna(how='all', axis=1).columns
            cols_to_add = np.setdiff1d(train_cols,test_cols)
            
            # Add to the test dict those features
            for treat in ('Ionly', 'NGP', 'Ionly_RF', 'NGP_RF'):
                test_numb = len(test_dict['test'][fold][treat])
                df_prep = []
                for i in cols_to_add:
                    df_prep.append(min(test_dict['train'][fold][treat][i]))

                df = test_dict['test'][fold][treat].copy()
                df = pd.concat((test_dict['test'][fold][treat], 
                                pd.DataFrame((df_prep, ) * test_numb, 
                                             index=df.index, columns=cols_to_add)), axis=1)
                test_dict['test'][fold][treat] = df[
                    test_dict['train'][fold][treat].columns]

            fold = fold+1

        #ds['iter_fold_splits'][n_iter+1] = test_dict
        end = perf_counter()
        print(name, n_iter+1, f' iteration done! took {(end - start):.3f} s')
    
    end2 = perf_counter()
    print(name, f' dataset done! took {(end2 - start2):.3f} s!!')

In [ ]:
# Making the folds for each iteration of cross-validation and pre-treating for the HD dataset 
np.random.seed(214)
iter_num = 20

name, ds = 'HD', datasets['HD']

# Split the dataset into an external training and testing group
train_data, test_data, train_target, test_target = train_test_split(ds['data'], ds['target'], test_size=0.3)

ds['train'] = {}
ds['train']['data'] = train_data
ds['train']['target'] = train_target

ds['test'] = {}
ds['test']['data'] = test_data
ds['test']['target'] = test_target

# Treat each group independently
start = perf_counter()
compute_transf_iterations(ds['train'], norm_ref=None, train=True, max_iter=2)
compute_transf_iterations(ds['test'], norm_ref=None, train=False, max_iter=2)

# Drop features that do not appear in the training set and add features that do not appear in the test data but appear
# in the training data to the test data with the minimum value of said feature in the training data 
train_cols = ds['train']['data'].dropna(how='all', axis=1).columns
test_cols = ds['test']['data'].dropna(how='all', axis=1).columns
cols_to_add = np.setdiff1d(train_cols,test_cols)

for treat in ('Ionly', 'NGP', 'Ionly_RF', 'NGP_RF'):
    test_numb = len(ds['test'][treat])
    df_prep = []
    for i in cols_to_add:
        df_prep.append(min(ds['train'][treat][i]))

    df = ds['test'][treat].copy()
    df = pd.concat((ds['test'][treat], pd.DataFrame((df_prep, ) * test_numb, index=df.index, columns=cols_to_add)), axis=1)
    ds['test'][treat] = df[ds['train'][treat].columns] # Same order as training set

end = perf_counter()
print(name, f' main test and training data pre-treatment done! took {(end - start):.3f} s')

# Now, from the training set, generate the folds for every cross-validation iteration and pre-treat each of them
# with each pre-treatment
np.random.seed(16)
# Where it will be stored
ds['iter_fold_splits'] = {}

# Each Iteration
for n_iter in range(iter_num):
    
    # Generate each fold of training and test sets for each iteration, treat them and store them
    n_fold = 5
    kf = StratifiedKFold(n_fold, shuffle=True, random_state=307*(n_iter+1))
    ds['iter_fold_splits'][n_iter+1] = {}
    test_dict = ds['iter_fold_splits'][n_iter+1]
    test_dict['train'] = {}
    test_dict['test'] = {}
    fold = 1
    start = perf_counter()

    # Only use data from the main training set
    for train_index, test_index in kf.split(ds['train']['data'], ds['train']['target']):
        start2 = perf_counter()
        # Generating
        test_dict['train'][fold] = {}
        test_dict['test'][fold] = {}
        test_dict['train'][fold]['data'], test_dict['test'][fold]['data'] = ds['train']['data'].iloc[
            train_index, :], ds['train']['data'].iloc[test_index, :]
        test_dict['train'][fold]['target'], test_dict['test'][fold]['target'] = [
            ds['train']['target'][i] for i in train_index], [ds['train']['target'][i] for i in test_index]

        # Treating
        compute_transf_iterations(test_dict['train'][fold], norm_ref=None, train=True, max_iter=2)
        compute_transf_iterations(test_dict['test'][fold], norm_ref=None, train=False, max_iter=2)

        train_cols = test_dict['train'][fold]['data'].dropna(how='all', axis=1).columns
        test_cols = test_dict['test'][fold]['data'].dropna(how='all', axis=1).columns
        cols_to_add = np.setdiff1d(train_cols,test_cols)

        # An extra safe net that needs to be applied due to the earlier split into a static test and training group
        # This might lead to features that are present in 2 samples, for example, to have a chance of one being in the
        # test group and another in the training. Thus, in those case features had to be removed before RF_Imputation
        # (requires a feature to be present in at least two samples), so the set of features in the data from 1/5 min
        # imputation and RF imputation are different
        train_cols_rf = test_dict['train'][fold]['Ionly_RF'].dropna(how='all', axis=1).columns
        test_cols_rf = test_dict['test'][fold]['Ionly_RF'].dropna(how='all', axis=1).columns
        cols_to_add_rf = np.setdiff1d(train_cols_rf,test_cols_rf)

        for treat in ('Ionly', 'NGP', 'Ionly_RF', 'NGP_RF'):
            test_numb = len(test_dict['test'][fold][treat])
            df_prep = []
            
            if treat.endswith('_RF'): # If RF imputation add the respective features to add
                for i in cols_to_add_rf:
                    df_prep.append(min(test_dict['train'][fold][treat][i]))
            else: # If 1/5 min imputation add the respective features to add
                for i in cols_to_add:
                    df_prep.append(min(test_dict['train'][fold][treat][i]))

            df = test_dict['test'][fold][treat].copy()
            if treat.endswith('_RF'):
                df = pd.concat((test_dict['test'][fold][treat], 
                                pd.DataFrame((df_prep, ) * test_numb, 
                                             index=df.index, columns=cols_to_add_rf)), axis=1)
            else:
                df = pd.concat((test_dict['test'][fold][treat], 
                                pd.DataFrame((df_prep, ) * test_numb, 
                                             index=df.index, columns=cols_to_add)), axis=1)
            test_dict['test'][fold][treat] = df[
                test_dict['train'][fold][treat].columns] # Order the 

        end2 = perf_counter()
        print('Fold number', fold, f' done! took {(end2 - start2):.3f} s!!')
        fold = fold+1

    #ds['iter_fold_splits'][n_iter+1] = test_dict
    end = perf_counter()
    print(name, n_iter+1, f' iteration done! took {(end - start):.3f} s')

In [ ]:
# Transform float columns (names of features) into strings so scikit-learn functions can handle them better
for name, ds in datasets.items():
    print(name)
    for treat in ('data', 'Ionly', 'NGP', 'Ionly_RF', 'NGP_RF'):
        ds[treat].columns = [str(col) for col in ds[treat].columns]
        # Only HD has train and test keys
        if name == 'HD':
            ds['train'][treat].columns = [str(col) for col in ds['train'][treat].columns]
            ds['test'][treat].columns = [str(col) for col in ds['test'][treat].columns]

    ds_iter = ds['iter_fold_splits']
    for itr in range(len(ds_iter.keys())):
        for fold in ds_iter[itr+1]['train'].keys():
            for treat in ('data', 'Ionly', 'NGP', 'Ionly_RF', 'NGP_RF'):
                ds_iter[itr+1]['train'][fold][treat].columns = [
                    str(col) for col in ds_iter[itr+1]['train'][fold][treat].columns]
                ds_iter[itr+1]['test'][fold][treat].columns = [
                    str(col) for col in ds_iter[itr+1]['test'][fold][treat].columns]

### Generate json files

In [ ]:
GENERATE = True

In [ ]:
if GENERATE:
    # ensure dir exists
    path = Path.cwd() / "store_files"
    path.mkdir(parents=True, exist_ok=True)

    storepath = Path.cwd() / "store_files" / 'processed_data.h5'

    store = pd.HDFStore(storepath, complevel=9, complib="blosc:blosclz")
    #pd.set_option('io.hdf.default_format','table')

    # keep json serializable values and store dataFrames in HDF store

    serializable = {}
    # Store in and h5 store the pandas dataframes created and in json file the rest
    # Since we have a lot of nested dicts in 'iter_fold_splits', the save and load the files back up code is a bit complex
    # Probably not the best way but functional
    # 'AA_' and 'TTS_' are used as special delimiters for the iter_fold_splits and train and test sets keys in the dict
    # Since they have the nested dicts to call on them when reading back the files
    for dskey, dataset in datasets.items():
        serializable[dskey] = {}
        for key, value in dataset.items():
            #print(dskey, key)
            
            # Save DataFrames
            if isinstance(value, pd.DataFrame):
                storekey = dskey + '_' + key
                #print('-----', storekey)
                store[storekey] = value
                serializable[dskey][key] = f"INSTORE_{storekey}"
            
            # Save DataFrames in nested dicts of the iter fold splits
            elif key == 'iter_fold_splits':
                for iteration, i in value.items():
                    for group, n in i.items():
                        for fold, j in n.items():
                            #print(j)
                            for treat, dfs in j.items():
                                storekey = dskey + '_' + key + 'AA_' + str(iteration) + '_' + group + '_' + str(fold) + '_' + treat
                                if treat == 'target':
                                    serializable[dskey][storekey] = dfs
                                #print(df)
                                else:
                                    store[storekey] = dfs
                                    serializable[dskey][storekey] = f"INSTORE_{storekey}"
            
            # Save DataFrames in nested dict for train and test group of the HD
            elif key in ('train','test'):
                for treat, dfs in value.items():
                    storekey = dskey + '_' + key + 'TTS_' + treat
                    if treat == 'target':
                        serializable[dskey][storekey] = dfs
                    #print(df)
                    else:
                        store[storekey] = dfs
                        serializable[dskey][storekey] = f"INSTORE_{storekey}"
            else:
                serializable[dskey][key] = value
    store.close()


    path = path / 'processed_data.json'
    with open(path, "w", encoding='utf8') as write_file:
        json.dump(serializable, write_file)

    #serializable

#### Reading back the files if you want

In [ ]:
# Where the datasets are
path = Path.cwd() / "store_files" / 'processed_data.json'
storepath = Path.cwd() / "store_files" / 'processed_data.h5'
with pd.HDFStore(storepath) as store:
    
    # Read into a dictionary not DataFrame data
    with open(path, encoding='utf8') as read_file:
        datasets = json.load(read_file)
    
    # Add DataFrame data to dict
    for dskey, dataset in datasets.items():
        dataset['iter_fold_splits'] = {}
        if dskey == 'HD':
            dataset['train'] = {}
            dataset['test'] = {}
        for key in dataset:
            # Created right before
            if 'iter_fold_splits' == key:
                continue
            value = dataset[key]
            if isinstance(value, str) and value.startswith("INSTORE"):
                storekey = value.split("_", 1)[1]
                #print(storekey)
                # Load the data from 'iter_fold_splits' carefully restoring the nested dictionaries
                if len(storekey.split("AA_")) > 1: # This separation was made to identify the 'iter_fold_splits' data
                    dictkeys = (storekey.split("AA_")[1]).split('_',3)
                    # Create nested dicts
                    if int(dictkeys[0]) not in dataset['iter_fold_splits'].keys():
                        dataset['iter_fold_splits'][int(dictkeys[0])] = {}
                    if dictkeys[1] not in dataset['iter_fold_splits'][int(dictkeys[0])].keys():
                        dataset['iter_fold_splits'][int(dictkeys[0])][dictkeys[1]] = {}
                    if int(dictkeys[2]) not in dataset['iter_fold_splits'][int(dictkeys[0])][dictkeys[1]].keys():
                        dataset['iter_fold_splits'][int(dictkeys[0])][dictkeys[1]][int(dictkeys[2])] = {}
                    dataset['iter_fold_splits'][int(dictkeys[0])][dictkeys[1]][int(dictkeys[2])][dictkeys[3]] = store[storekey]
                
                # Load the data from 'train' and 'test' from HD dataset keys carefully restoring the nested dictionaries
                elif len(storekey.split("TTS_")) > 1:
                    dictkeys = ((storekey.split("TTS_")[0]).split('_')[-1], storekey.split("TTS_")[1])#.split('_',2)
                    dataset[dictkeys[0]][dictkeys[1]] = store[storekey]
                # Normal DataFrames
                else:
                    dataset[key] = store[storekey]

            # convert colors to tuples, since they are read as lists from json file
            elif key == 'label_colors':
                dataset[key] = {lbl: tuple(c) for lbl, c in value.items()}
            elif key == 'sample_colors':
                dataset[key] = [tuple(c) for c in value]
            elif key.endswith('target') and key.startswith(dskey):
                if len(key.split("AA_")) > 1: 
                    dictkeys = ((key.split("_", 1)[1]).split("AA_")[1]).split('_',3)
                    dataset['iter_fold_splits'][int(dictkeys[0])][dictkeys[1]][int(dictkeys[2])][dictkeys[3]] = value
                else:
                    dictkeys = ((key.split("TTS_")[0]).split('_')[-1], key.split("TTS_")[1])#.split('_',2)
                    dataset[dictkeys[0]][dictkeys[1]] = value

# Remove extra keys
for name, ds in datasets.items():
    keys_to_remove = [keys for keys in ds.keys() if keys.startswith(name)]
    for key in keys_to_remove:
        ds.pop(key)
